In [22]:
import pandas as pd
import folium, json
import warnings
warnings.filterwarnings('ignore')

In [23]:
df = pd.read_csv("data/Seoul_Cinema.csv")
df.drop(['자치구별(1)'], axis=1, inplace=True)
df_T = df.transpose()
df_T.rename(columns=df_T.iloc[0], inplace=True)
df_T = df_T.drop(df_T.index[0])
df_T.index.name = '년도'
df_T.rename(columns={'자치구별(2)':'구별'}, inplace=True)
df_T.head()

,구별,소계,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,...,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구
년도,,,,,,,,,,,,,,,,,,,,,
1993,스크린수 (개),181,27,-,16,1,12,13,3,3,...,1,9,-,11,5,7,5,23,7,7
1994,스크린수 (개),181,27,-,16,1,12,13,3,3,...,1,9,-,11,5,7,5,23,7,7
1995,스크린수 (개),183,29,12,1,1,4,11,3,4,...,-,9,1,11,3,6,5,30,7,5
1996,스크린수 (개),179,31,18,2,1,4,8,3,3,...,-,6,1,11,2,6,7,24,7,5
1997,스크린수 (개),173,27,17,3,1,2,8,3,3,...,-,5,1,11,2,5,7,26,6,7


In [24]:
df_2020 = df_T[-3:]
df_2020.set_index("구별", inplace=True)
df_2020.drop(['소계'], axis=1, inplace=True)
df_2020.index.name = ""
df_2020.head()

,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,...,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구
,,,,,,,,,,,,,,,,,,,,,
개소수 (개소),9,9,2,2,5,2,3,2,3,1,...,5,3,2,3,2,2,2,9,3,3
스크린수 (개),28,42,28,17,38,15,22,8,21,5,...,35,25,14,28,8,13,12,54,38,26
좌석수 (개),4340,7704,5361,2887,5712,2822,3064,1238,2516,680,...,5849,3358,2660,5308,978,2181,1653,8208,7017,43


In [25]:
df_2020_T = df_2020.transpose()
df_2020_T.fillna(0)
df_2020_T.to_csv("data/Seoul_Cinema_Choropleth.csv")
df_2020_T.reset_index(inplace=True)
df_2020_T.rename(columns={'index': '구별'}, inplace=True)
df_2020_T.set_index("구별", inplace=True)
df_2020_T.columns = ['개소수', '스크린수', '좌석수']
df_2020_T = df_2020_T.astype(int)
df_2020_T.head(25)

,개소수,스크린수,좌석수
구별,,,
종로구,9,28,4340
중구,9,42,7704
용산구,2,28,5361
성동구,2,17,2887
광진구,5,38,5712
동대문구,2,15,2822
중랑구,3,22,3064
성북구,2,8,1238
강북구,3,21,2516


In [26]:
filename = 'data/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

In [27]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str, 
    data=df_2020_T.스크린수,
    columns=[df_2020_T.index, df_2020_T.스크린수],
    fill_color='PuRd',
    key_on='feature.id'
).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">영화관 스크린수</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

In [28]:
map = folium.Map(location=[37.5502, 126.982],
                 zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str,
    data=df_2020_T.좌석수,
    columns=[df_2020_T.index, df_2020_T.좌석수],
    fill_color='PuRd',
    key_on='feature.id'
).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">영화관 좌석수</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map


In [29]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

map.choropleth(geo_data=geo_str,
               data=df_2020_T.좌석수,
               columns=[df_2020_T.index, df_2020_T.좌석수],
               fill_color='PuRd',  # PuRd, YlGnBu
               key_on='feature.id')
title_html = '<h3 align="center" style="font-size:20px">영화관 좌석수</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map


In [30]:
map.save("Map/Seoul_Seat.html")

In [31]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)
folium.Choropleth(
    geo_data=geo_str, 
    data=df_2020_T.스크린수,
    columns=[df_2020_T.index, df_2020_T.스크린수],
    fill_color='PuRd',
    key_on='feature.id'
).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">영화관 스크린수</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

In [32]:
# 스크린수 MAP html처리
map.save("./Map/Seoul_Screen.html")